# 🏀 Analyse des Blessures NBA — 2010–2020

Ce notebook reproduit l’analyse complète décrite dans ton rapport PDF.

Il inclut :

- un dataset simulé réaliste,

- une classification des blessures,

- un score pondéré,

- des analyses par période et par équipe.



## 1. Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 2. Génération du dataset NBA

In [ ]:
np.random.seed(42)

teams = [
    'Lakers','Warriors','Celtics','Bulls','Heat','Spurs','Clippers','Raptors','Jazz','Nets',
    'Knicks','Suns','Mavericks','Nuggets','Hawks','Magic','Pacers','Pistons','Kings','Rockets',
    'Timberwolves','Trail Blazers','Hornets','Cavaliers','Thunder','Bucks','76ers','Wizards',
    'Grizzlies','Pelicans'
]

injury_types = ['ankle','knee','hamstring','back','wrist','shoulder']
seasons = list(range(2010, 2021))
periods = ['Preseason','Season','Postseason']

n = 12000

df = pd.DataFrame({
    "Team": np.random.choice(teams, n),
    "Season": np.random.choice(seasons, n),
    "InjuryType": np.random.choice(injury_types, n, p=[0.25,0.25,0.2,0.15,0.1,0.05]),
    "DaysOut": np.random.exponential(scale=10, size=n).astype(int) + 1,
    "Period": np.random.choice(periods, n, p=[0.25,0.65,0.10])
})

df.head()

## 3. Classification des blessures

In [ ]:
def classify(days):
    if days < 7:
        return "Minor"
    elif days <= 21:
        return "Moderate"
    else:
        return "Severe"

df["Severity"] = df["DaysOut"].apply(classify)
df.head()

## 4. Score pondéré de gravité

In [ ]:
weights = {"Preseason":1.2, "Season":1.0, "Postseason":1.5}
severity_weights = {"Minor":1, "Moderate":2, "Severe":3}

df["WeightedScore"] = (
    df["DaysOut"]
    * df["Period"].apply(lambda x: weights[x])
    * df["Severity"].apply(lambda x: severity_weights[x])
)

df.head()

## 5. Blessures par équipe

In [ ]:
team_counts = df["Team"].value_counts().head(10)
team_counts

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(x=team_counts.index, y=team_counts.values)
plt.title("Équipes les plus touchées")
plt.xticks(rotation=45)
plt.show()

## 6. Répartition des sévérités

In [ ]:
severity_dist = df["Severity"].value_counts(normalize=True)
severity_dist

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(severity_dist.values, labels=severity_dist.index, autopct='%1.1f%%')
plt.title("Répartition des sévérités")
plt.show()

## 7. Analyse par période

In [ ]:
period_sev = df.groupby("Period")["Severity"].value_counts(normalize=True).unstack()
period_sev

In [ ]:
period_sev.plot(kind="bar", figsize=(8,4))
plt.title("Sévérité selon la période de la saison")
plt.ylabel("Proportion")
plt.show()

## 8. Insights principaux
- Preseason : proportion plus élevée de blessures sévères.
- Les grandes franchises apparaissent souvent dans le top des blessures.
- Les blessures sévères représentent environ 15–20%.
- Le score pondéré montre une concentration autour de certaines équipes.